In [141]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [116]:
# Load dataset
rentals = pd.read_csv('rental_info.csv')

In [117]:
# Explore dataset

In [118]:
rentals.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [119]:
# Check data types and shape of the dataset
rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


In [120]:
# Check for missing values
rentals.isna().sum()

rental_date         0
return_date         0
amount              0
release_year        0
rental_rate         0
length              0
replacement_cost    0
special_features    0
NC-17               0
PG                  0
PG-13               0
R                   0
amount_2            0
length_2            0
rental_rate_2       0
dtype: int64

In [121]:
# Data preprocessing 

In [122]:
## Target variable 

### Convert columns containing dates to datetime
rentals['rental_date'] = pd.to_datetime(rentals['rental_date'])
rentals['return_date'] = pd.to_datetime(rentals['return_date'])

### Calculate rental length (target for prediction) and store in new column
rentals['rental_length_days'] = (rentals['return_date'] - rentals['rental_date']).dt.days

In [123]:
## Features

### Create dummy variables for column (feature) 'special_features'
rentals['deleted_scenes'] = np.where(rentals['special_features'].str.contains('Deleted Scenes'), 1,0)
rentals['behind_scenes'] = np.where(rentals['special_features']   .str.contains('Behind the Scenes'), 1,0)

In [124]:
# Split the dataset into feature and target sets
X = rentals.drop(['rental_date', 'return_date', 'rental_length_days', "special_features"], axis=1)
y = rentals['rental_length_days']

In [ ]:
# 1 - Run a linear regression model (OLS)

In [125]:
# Determine which features are most predictive using a lasso model
lasso = Lasso(alpha=0.3, random_state=9)
lasso_coef = lasso.fit(X, y).coef_
features = X.columns

features_coef = pd.DataFrame({'Features': features, 'Lasso_coef': lasso_coef})
features_coef

,Features,Lasso_coef
0,amount,0.587568
1,release_year,0.000000
2,rental_rate,-0.000000
3,length,0.000000
4,replacement_cost,-0.000000
5,NC-17,0.000000
6,PG,0.000000
7,PG-13,0.000000
8,R,-0.000000
9,amount_2,0.043265


In [126]:
# Update feature set keeping only features with positive coefficients
X_lasso = X[['amount', 'amount_2', 'length_2']]

In [127]:
# Split dataset into train and test sets
X_lasso_train, X_lasso_test, y_train, y_test = train_test_split(X_lasso, y, test_size=0.2, random_state=9)

In [137]:
# Run the OLS model on the selected features and caculate MSE using cross-validation with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=9)
ols = LinearRegression()
cv_results = cross_val_score(ols, X_lasso, y, cv=kf, scoring='neg_mean_squared_error')
ols_mse = abs(cv_results.mean())
print(ols_mse)

4.79162998317893


In [131]:
# 2 - Run a Random Forest Regressor

In [132]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [136]:
# Create a Random Forest regressor model
rf = RandomForestRegressor()

In [145]:
# Determine optimal Random Forest model hyperparameters 
param_dist = {'n_estimators': np.arange(1,101,1), 'max_depth':np.arange(1,11,1)}
grid_rf = RandomizedSearchCV(rf, param_distributions=param_dist, cv=5, random_state=9)
grid_rf.fit(X_train, y_train)
best_hyperparams = grid_rf.best_params_
print(best_hyperparams)

{'n_estimators': 51, 'max_depth': 10}


In [148]:
# Run the Random Forest Regressor model with the optimal hyperparameters caculate MSE
rf_optimised = RandomForestRegressor(n_estimators=51, max_depth=10, random_state=9)
rf_optimised.fit(X_train, y_train)
y_pred_rf = rf_optimised.predict(X_test)
rf_mse = MSE(y_test, y_pred_rf)
print(rf_mse)

2.225667528098759
